<a href="https://colab.research.google.com/github/MatheusFranklinLH/SpellChecker/blob/main/SpellChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open("artigos.txt", "r") as f: #Read file artigos.txt
  articles = f.read() 

print(articles[:500]) #Corpus in portuguese

In [ ]:
print(len(articles))
len("Hello")

In [ ]:
text_example = "Hello, how are you?" # Just a example
tokens = text_example.split()
print(tokens)
len(tokens)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
splited_words = nltk.tokenize.word_tokenize(text_example)
print(splited_words)

In [ ]:
'word'.isalpha() #String methods: return true if all characters are alphabetics (letters)

In [ ]:
def split_words(token_list): # Returns only word tokens
  word_list = []
  for token in token_list:
    if token.isalpha():
      word_list.append(token)
  return word_list

print(split_words(splited_words))

In [ ]:
token_list = nltk.tokenize.word_tokenize(articles)
words_list = split_words(token_list)
print(f"Number of words in corpus: {len(words_list)}  ")

In [ ]:
print(words_list[:5])

In [17]:
def normalize(words_list):
  normalized_list = []
  for word in words_list:
    normalized_list.append(word.lower()) # lower turns every letter to lowercase
  return normalized_list

In [ ]:
normalized_list = normalize(words_list)
print(normalized_list[:5])

In [ ]:
set([1,2,3,3,3,4,5,6]) # Eliminate repetition

In [20]:
print(f"Actual amount of words: {len(set(normalized_list))}")

Actual amount of words: 17652


In [ ]:
list = "lgic"
(list[:1], list[1:])

In [ ]:
word_example = "lgic" #Logic misspelled

def insert_letters(slices):
  new_words = []
  letters = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for L, R in slices:
    for letter in letters:
      new_words.append(L+letter+R)
  return new_words

def word_generator(word):
  slices = []
  for i in range(len(word)+1):
    slices.append((word[:i],word[i:]))
  generated_words = insert_letters(slices)
  return generated_words

gen_words = word_generator(word_example)
print(gen_words)

In [23]:
def spell_checker(word):
  gen_words = word_generator(word)
  correct_word = max(gen_words, key=probability) #Return the most likely word
  return correct_word
  

In [ ]:
frequency = nltk.FreqDist(normalized_list)
total_amount = len(normalized_list)
frequency.most_common(10)

In [ ]:
def probability(gen_word):
  return frequency[gen_word]/total_amount

probability("logica")

In [ ]:
spell_checker("lgica") # lógica means logic

In [ ]:
def delete_letters(slices):
  new_words = []
  for L, R in slices:
      new_words.append(L+R[1:])
  return new_words

def change_letters(slices):
  new_words = []
  letters = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for L, R in slices:
    for letter in letters:
      new_words.append(L+letter+R[1:])
  return new_words

def reverse_letters(slices):
  new_words = []
  for L, R in slices:
    if(len(R)>=2):
      new_words.append(L+R[1]+R[0]+R[2:])
  return new_words

def word_generator(word):
  slices = []
  for i in range(len(word)+1):
    slices.append((word[:i],word[i:]))
  generated_words = insert_letters(slices)
  generated_words += delete_letters(slices)
  generated_words += change_letters(slices)
  generated_words += reverse_letters(slices)
  return generated_words

gen_words = word_generator("lgóica")
print(gen_words)

In [ ]:
def create_test_data(file_name): #The test samples are arranged in lines where first comes the correct word and then misspeled
  test_word_list = []
  f = open(file_name, "r")
  for line in f:
    correct, wrong = line.split()
    test_word_list.append((correct, wrong))
  f.close()
  return test_word_list

test_list = create_test_data("palavras.txt")
print(test_list)

In [52]:
def rater(tests, vocabulary):
  total_words = len(tests)
  hit = 0
  unknown = 0
  for correct, wrong in tests:
    if spell_checker(wrong) == correct:
      hit += 1
    else:
      unknown += (correct not in vocabulary)
  hit_rate = round(hit * 100 / total_words, 2)
  unknown_rate = round(unknown * 100 / total_words, 2)
  print(f"Hit rate: {hit_rate}% of {total_words} words, unknown words rate: {unknown_rate}%")

vocabulary = set(normalized_list)
rater(test_list, vocabulary)

Hit rate: 76.34% of 186 words, unknown words rate: 6.99%


In [ ]:
word = "lóiigica"

def turbo_generator(gen_words):
  new_words = []
  for word in gen_words:
    new_words += word_generator(word)
  return new_words

g_words = turbo_generator(word_generator(word))
"lógica" in g_words

In [ ]:
len(g_words) # Too much samples for only one word

In [ ]:
def new_spell_checker(word, vocabulary):
  gen_words = word_generator(word)
  turbo_gen_words = turbo_generator(gen_words)
  words = set(gen_words + turbo_gen_words)
  candidates = [word]
  for w in words:
    if w in vocabulary:
      candidates.append(w)
  #print(len(candidates))
  correct_word = max(candidates, key=probability) #Return the most likely word
  return correct_word

new_spell_checker("lóiigica", vocabulary)

In [ ]:
def rater(tests, vocabulary):
  total_words = len(tests)
  hit = 0
  unknown = 0
  for correct, wrong in tests:
    checked_word = new_spell_checker(wrong, vocabulary)
    unknown += (correct not in vocabulary)
    if  checked_word == correct:
      hit += 1
    #else:
      #print(wrong + " - " + spell_checker(wrong) + " - " + checked_word)
  hit_rate = round(hit * 100 / total_words, 2)
  unknown_rate = round(unknown * 100 / total_words, 2)
  print(f"Hit rate: {hit_rate}% of {total_words} words, unknown words rate: {unknown_rate}%")

vocabulary = set(normalized_list)
rater(test_list, vocabulary)

##Turbo checker had a worse perform than the spell checker

In [66]:
word = "lgica"
print(new_spell_checker(word, vocabulary))
print(spell_checker(word))

fica
lógica
